In [5]:
import pandas as pd
import numpy as np
import sys

In [33]:
py_version = '{}.{}.{}'.format(sys.version_info.major, sys.version_info.minor, sys.version_info.micro)
print('Pandas Version: {}\nNumpy Version : {}\nPython Version: {} '.format(pd.__version__, np.__version__, py_version))

Pandas Version: 0.22.0
Numpy Version : 1.14.0
Python Version: 3.6.4 


In [6]:
data = np.array([['','Market','Product Code','Week','Sales','Units'],
            ['Total Customers',123,1,500,400],
            ['Total Customers',123,2,400,320],
            ['Major Customer 1',123,1,100,220],
            ['Major Customer 1',123,2,230,230],
            ['Major Customer 2',123,1,130,30],
            ['Major Customer 2',123,2,20,10],
            ['Total Customers',456,1,500,400],
            ['Total Customers',456,2,400,320],
            ['Major Customer 1',456,1,100,220],
            ['Major Customer 1',456,2,230,230],
            ['Major Customer 2',456,1,130,30],
            ['Major Customer 2',456,2,20,10]])

df =pd.DataFrame(data)

In [80]:
df

,0,1,2,3,4,5
0,,Market,Product Code,Week,Sales,Units
1,Total Customers,123,1,500,400,NaN
2,Total Customers,123,2,400,320,NaN
3,Major Customer 1,123,1,100,220,NaN
4,Major Customer 1,123,2,230,230,NaN
5,Major Customer 2,123,1,130,30,NaN
6,Major Customer 2,123,2,20,10,NaN
7,Total Customers,456,1,500,400,NaN
8,Total Customers,456,2,400,320,NaN
9,Major Customer 1,456,1,100,220,NaN


In [89]:
'Market' in df.columns

False

In [83]:
df1 = pd.DataFrame([pd.Series(l) for l in data])
df1

,0,1,2,3,4,5
0,,Market,Product Code,Week,Sales,Units
1,Total Customers,123,1,500,400,NaN
2,Total Customers,123,2,400,320,NaN
3,Major Customer 1,123,1,100,220,NaN
4,Major Customer 1,123,2,230,230,NaN
5,Major Customer 2,123,1,130,30,NaN
6,Major Customer 2,123,2,20,10,NaN
7,Total Customers,456,1,500,400,NaN
8,Total Customers,456,2,400,320,NaN
9,Major Customer 1,456,1,100,220,NaN


In [84]:
'Market' in df1.columns

False

In [85]:

df1.columns = df1.iloc[0]

df2 = df1.reindex(df1.index.drop(0))

df2

,,Market,Product Code,Week,Sales,Units
1,Total Customers,123,1,500,400,NaN
2,Total Customers,123,2,400,320,NaN
3,Major Customer 1,123,1,100,220,NaN
4,Major Customer 1,123,2,230,230,NaN
5,Major Customer 2,123,1,130,30,NaN
6,Major Customer 2,123,2,20,10,NaN
7,Total Customers,456,1,500,400,NaN
8,Total Customers,456,2,400,320,NaN
9,Major Customer 1,456,1,100,220,NaN
10,Major Customer 1,456,2,230,230,NaN


In [86]:
'Market' in df2.columns

True

In [87]:
# save columns to variable
df_cols = df2.columns[1:].tolist()

# Drop unit column
df3 = df2.drop('Units', axis=1)

# reset column headers
df3.columns = df_cols



In [88]:
df3

,Market,Product Code,Week,Sales,Units
1,Total Customers,123,1,500,400
2,Total Customers,123,2,400,320
3,Major Customer 1,123,1,100,220
4,Major Customer 1,123,2,230,230
5,Major Customer 2,123,1,130,30
6,Major Customer 2,123,2,20,10
7,Total Customers,456,1,500,400
8,Total Customers,456,2,400,320
9,Major Customer 1,456,1,100,220
10,Major Customer 1,456,2,230,230


In [102]:
# Sorted view.  We  now understand where the Remaining Customers row fits.
df3.sort_values(['Week', 'Product Code' ])

,Market,Product Code,Week,Sales,Units
1,Total Customers,123,1,500,400
3,Major Customer 1,123,1,100,220
5,Major Customer 2,123,1,130,30
7,Total Customers,456,1,500,400
9,Major Customer 1,456,1,100,220
11,Major Customer 2,456,1,130,30
2,Total Customers,123,2,400,320
4,Major Customer 1,123,2,230,230
6,Major Customer 2,123,2,20,10
8,Total Customers,456,2,400,320


In [113]:
# Step 4a: compute total sales and units by product by week
#~df3.Market.str.contains("Total") >> In english: Markets that do not contain Total

df4a = df3[~df3.Market.str.contains("Total")].groupby(['Product Code', 'Week']).sum()[['Sales', 'Units']]
df4a = df4a.reset_index()
df4a

,Product Code,Week,Sales,Units
0,123,1,230,250
1,123,2,250,240
2,456,1,230,250
3,456,2,250,240


In [118]:
# Step 4b: prepare totals data frame to subtract differences
df4b = df3[df3.Market.str.contains("Total")][['Product Code', 'Week', 'Sales', 'Units']].reset_index(drop=True)
df4b

,Product Code,Week,Sales,Units
0,123,1,500,400
1,123,2,400,320
2,456,1,500,400
3,456,2,400,320


In [120]:
# step 4c: subtract differences.  We can do this because the index 0-3 align ! 
df4b[['Sales', 'Units']] - df4a[['Sales', 'Units']]

,Sales,Units
0,270,150
1,150,80
2,270,150
3,150,80


In [128]:
df5 = pd.concat([df4b[['Product Code', 'Week']], df4b[['Sales', 'Units']] - df4a[['Sales', 'Units']]], axis=1)
df5['Market'] ='Remaining Customers'
df5

,Product Code,Week,Sales,Units,Market
0,123,1,270,150,Remaining Customers
1,123,2,150,80,Remaining Customers
2,456,1,270,150,Remaining Customers
3,456,2,150,80,Remaining Customers


In [134]:
col_order = ['Market', 'Product Code', 'Week', 'Sales', 'Units']
pd.concat([df3,df5]).sort_values(['Product Code', 'Week', 'Market'])[col_order]

,Market,Product Code,Week,Sales,Units
3,Major Customer 1,123,1,100,220
5,Major Customer 2,123,1,130,30
0,Remaining Customers,123,1,270,150
1,Total Customers,123,1,500,400
4,Major Customer 1,123,2,230,230
6,Major Customer 2,123,2,20,10
1,Remaining Customers,123,2,150,80
2,Total Customers,123,2,400,320
9,Major Customer 1,456,1,100,220
11,Major Customer 2,456,1,130,30
